# Installing necessary Libraries

In [2]:
pip install emoji==1.6.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169295 sha256=ee0f05e91b7818c2c424427b6fa87fdd222afed9f0a870235405362a61dbfa89
  Stored in directory: /root/.cache/pip/wheels/3d/c9/af/02caa5725634f27f4e2e43852f67fc9069d014038b236a827e
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.12.1
    Uninstalling emoji-2.12.1:
      Successfully uninstalled emoji-2.12.1
Note: you may need to restart the kernel to use updated packages.


# All the necessary imports

In [17]:

import json
import emoji
from collections import Counter
import pandas as pd
import re
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'Jewish', 'Arab', 'Asian', 'None', 'Caucasian', 'Women', 'Islam', 'Refugee', 'African', 'Hispanic', 'Homosexual'}


# Data Preprocessing

In [ ]:
## Data Processing

with open('/kaggle/input/data111/dataset.json', 'r') as file:
    data = json.load(file)



## MAKING LABELS


FINAL_LABEL_LIST =[]
for k in list(data.keys()):
  # Extract labels
  label_list = [item['label'] for item in data[k]['annotators']]

  # Count occurrences of each label
  label_counts = Counter(label_list)

  # Calculate total number of labels
  total_labels = len(label_list)

  # Determine the label with the highest frequency
  most_common_label, count = label_counts.most_common(1)[0]

  # Check if the highest frequency label occurs more than 50% of the time
  if count / total_labels > 0.5:
      assigned_label = most_common_label
  else:
      assigned_label = None
  FINAL_LABEL_LIST.append(assigned_label)


Counter(FINAL_LABEL_LIST)

input_data= []
rationales = []
post_ids=[]
for k in list(data.keys()):
  # print(k)
  # print(data[k]['rationales'])
  if k =='24439295_gab':
    rationales.append([])
  else:
    rationales_array = np.array(data[k]['rationales'])
    averaged_rationales = np.mean(rationales_array, axis=0)
    averaged_rationales_list = averaged_rationales.tolist()
    if type(averaged_rationales_list) ==list:
      finalized_rationales = [1 if value > 0.5 else 0 for value in averaged_rationales_list]
    else:
      finalized_rationales =[0]*len(data[k]['post_tokens'])

    rationales.append(finalized_rationales)
  input_data.append(' '.join(data[k]['post_tokens']))
  post_ids.append(data[k]['post_id'])

len(input_data), len(rationales), len(FINAL_LABEL_LIST)

# Sample data
# Function to process the data
FINAL_TARGET_LIST = []
def get_majority_targets(data):
    target_count = {}
    total_entries = len(data)

    # Count the occurrences of each target
    for entry in data:
        for target in entry['target']:
            if target in target_count:
                target_count[target] += 1
            else:
                target_count[target] = 1

    # Determine which targets appear in more than 50% of the entries
    majority_targets = [target for target, count in target_count.items() if count / total_entries > 0.5]

    return majority_targets

for k in list(data.keys()):
  # print(data[k]['annotators'])
  majority_targets = get_majority_targets(data[k]['annotators'])
  if majority_targets==[]:
    majority_targets=['None']
  FINAL_TARGET_LIST.append(majority_targets)

#print("Targets appearing in more than 50% of the entries:", FINAL_TARGET_LIST)


df= pd.DataFrame()
df['post_ids']=post_ids
df['input_text']= input_data
df['rationales'] = rationales
df['label']= FINAL_LABEL_LIST
df['Final_target']= FINAL_TARGET_LIST
df.head()

df = df.dropna(subset=['label'])
df

# def clean_html_tags_and_update_rationales(row):
#     """Remove HTML-like tags from tokens and update rationales accordingly."""
#     tokens = row['input_text'].split()
#     rationales = row['rationales']
#     cleaned_tokens = []
#     cleaned_rationales = []

#     for token, rationale in zip(tokens, rationales):
#         if not (token.startswith('<') and token.endswith('>')):
#             cleaned_tokens.append(token)
#             cleaned_rationales.append(rationale)

#     # Return a series with updated values
#     return pd.Series([cleaned_tokens, cleaned_rationales], index=['input_text', 'rationales'])

# # Apply the cleaning function to each row
# df[['input_text', 'rationales']] = df.apply(clean_html_tags_and_update_rationales, axis=1)

# # Since 'cleaned_input_text' is now a list of tokens, we need to join them back into strings
# df['input_text'] = df['input_text'].apply(' '.join)


# Function to transform labels to toxic non toxic
# def transform_label(label):
#     if label == 'normal':
#         return 'non-toxic'
#     elif label in ['hatespeech', 'offensive']:
#         return 'toxic'
#     else:
#         return label

# # Apply the transformation
# df['label'] = df['label'].apply(transform_label)



df['label'].value_counts()

final_communities_sel = ['African',
 'Islam',
 'Jewish',
 'Homosexual',
 'Women',
 'Refugee',
 'Arab',
 'Caucasian',
 'Asian',
 'Hispanic']
df.reset_index(inplace=True, drop=True)
final_target_information =[]
for i in range(len(df)) :
    temp = list(set(df['Final_target'][i])&set(final_communities_sel))
    if len(temp) == 0:
        final_target_information.append(['None'])
    else:
        final_target_information.append(temp)
df['Final_target']=final_target_information



all_values = []
for item in df['Final_target']:
    if isinstance(item, list):
        all_values.extend(item)
    else:
        all_values.append(item)  # This handles non-list items, just in case

# Get unique values from the list of all values
unique_values = set(all_values)

# Display the unique values
# print(unique_values)

# def deemojize_text(text):
#     return emoji.demojize(text,use_aliases=True)
# df['input_text'] = df['input_text'].apply(deemojize_text)

# df

# df.to_excel('tiktok_updated4.xlsx')

In [18]:
df

,post_ids,input_text,rationales,label,Final_target
0,1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",normal,[None]
1,1179063826874032128_twitter,we cannot continue calling ourselves feminists...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",normal,[None]
2,1178793830532956161_twitter,nawt yall niggers ignoring me,"[0, 0, 0, 0, 0]",normal,[African]
3,1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",hatespeech,[Asian]
4,1179085312976445440_twitter,this bitch in whataburger eating a burger with...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",hatespeech,"[Caucasian, Women]"
...,...,...,...,...,...
19224,9989999_gab,if ur still on twitter tell carlton i said his...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, ...",offensive,[None]
19225,9990225_gab,when i first got on here and said i hate trump...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",offensive,[African]
19226,9991681_gab,was macht der moslem wenn der zion gegen seine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",normal,[None]
19227,9992513_gab,it is awful look at world demographics asians ...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...",hatespeech,[Asian]


In [9]:
rationales =list(df['rationales'])

## MLM Training 

In [19]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler
from transformers import BertTokenizer, BertForMaskedLM, AdamW
from transformers import get_scheduler

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',truncate=True)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


device = torch.device("cuda")
model.to(device)

class TextDataset(Dataset):
    def __init__(self, texts, rationales, tokenizer, max_len=180, mask_percentage=0.15):
        self.tokenizer = tokenizer
        self.texts = texts
        self.rationales = rationales
        self.max_len = max_len
        self.mask_percentage = mask_percentage

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        rationale = self.rationales[idx]

        # Tokenization
        tokens = tokenizer.tokenize(text)
        input_ids = tokenizer.encode(tokens, add_special_tokens=True)

        # Apply rationale to each subword
        rationale_extended = []
        for word, rat in zip(text.split(), rationale):
            subwords = tokenizer.tokenize(word)
            rationale_extended.extend([rat] * len(subwords))
        rationale_extended = [0] + rationale_extended + [0]  # Adjust for [CLS] and [SEP]

        # Masking
        mask_positions = [i for i, r in enumerate(rationale_extended) if r == 1]
        num_to_mask = int(len(mask_positions) * self.mask_percentage)
        selected_masks = np.random.choice(mask_positions, num_to_mask, replace=False)
        labels = input_ids[:]  # Copy input_ids to labels

        # print("Before padding:")
        # print("Input IDs:", input_ids)
        # print("Labels:", labels)
        # print("Mask positions:", selected_masks)

        for i in selected_masks:
            input_ids[i] = tokenizer.mask_token_id

        # Padding
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:  # Pad if sequence is shorter than max_len
            input_ids.extend([tokenizer.pad_token_id] * padding_length)
            labels.extend([-100] * padding_length)

        return torch.tensor(input_ids), torch.tensor(labels)

# Parameters
texts = input_data
# rationales = list(df['rationales'])
# rationales
mask_percentage = 0.5# 50% of the tokens marked by 1 in the rationale will be masked

# Dataset and DataLoader
dataset = TextDataset(texts, rationales, tokenizer, max_len=180, mask_percentage=mask_percentage)
dataloader = DataLoader(dataset, batch_size=16, sampler=RandomSampler(dataset))

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
model.train()
for epoch in range(num_epochs):
    print(epoch)
    for batch in dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        model.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        #print(f"Epoch {epoch}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_bert')

# Evaluation
model.eval()
total_accuracy = 0
num_batches = 0

for batch in dataloader:
    inputs, labels = batch[0].to(device), batch[1].to(device)
    with torch.no_grad():
        outputs = model(inputs, labels=labels)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Exclude [CLS], [SEP], and [PAD] from accuracy calculation
        mask = (labels != -100) & (inputs != tokenizer.cls_token_id) & (inputs != tokenizer.sep_token_id)
        correct = (predictions == labels) & mask
        accuracy = correct.sum().item() / mask.sum().item() if mask.sum().item() > 0 else 0

        total_accuracy += accuracy
        num_batches += 1

average_accuracy = total_accuracy / num_batches
print(f"Average Accuracy: {average_accuracy}")

# Qualitative Analysis
# inputs, labels = next(iter(dataloader))
# with torch.no_grad():
#     outputs = model(inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1)

#     # Filter tokens for display
#     input_tokens = tokenizer.convert_ids_to_tokens(inputs[0])
#     prediction_tokens = [pred if (lab != -100 and inp not in [tokenizer.cls_token_id, tokenizer.sep_token_id]) else '[IGNORED]' for pred, lab, inp in zip(tokenizer.convert_ids_to_tokens(predictions[0]), labels[0], inputs[0])]

#     print("Input Tokens:      ", input_tokens)
#     print("Predicted Tokens:  ", prediction_tokens)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0
1
2
Average Accuracy: 0.9788632084692769


In [21]:
!zip -r bert_mlm_model.zip /kaggle/working/fine_tuned_bert

  adding: kaggle/working/fine_tuned_bert/ (stored 0%)
  adding: kaggle/working/fine_tuned_bert/generation_config.json (deflated 8%)
  adding: kaggle/working/fine_tuned_bert/model.safetensors (deflated 7%)
  adding: kaggle/working/fine_tuned_bert/config.json (deflated 47%)


## Classification Bert

In [22]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

# Ensure you have the necessary packages installed:
# pip install transformers torch sklearn

# Example labels and input data
labels = list(df['label'])
input_data = list(df['input_text'])

# Label Encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(input_data, encoded_labels, stratify=encoded_labels, test_size=0.2, random_state=42)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('/kaggle/working/fine_tuned_bert', num_labels=len(np.unique(train_labels)))


# Tokenization and input formatting
def tokenize_and_format(texts, labels):
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=180, return_tensors="pt")
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
    return dataset

train_dataset = tokenize_and_format(train_texts, train_labels)
val_dataset = tokenize_and_format(val_texts, val_labels)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Move model to GPU if available
device = torch.device("cuda")
model.to(device)

# Optimizer and Loss Function
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
def train(epoch):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = tuple(item for item in batch)
        inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device), 'labels': batch[2].to(device)}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch}, Training loss: {average_loss}')

# Evaluation function
def evaluate(loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = tuple(item for item in batch)
            inputs = {'input_ids': batch[0].to(device), 'attention_mask': batch[1].to(device)}
            labels = batch[2]
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    report = classification_report(true_labels, predictions, target_names=label_encoder.classes_, digits=3)
    print(report)

# Train the model
for epoch in range(1, 3):  # Number of epochs
    print(epoch)
    train(epoch)
    evaluate(val_loader)

# Evaluate the model
evaluate(val_loader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/fine_tuned_bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
Epoch 1, Training loss: 0.7793874450632043
              precision    recall  f1-score   support

  hatespeech      0.731     0.848     0.785      1187
      normal      0.760     0.756     0.758      1563
   offensive      0.596     0.496     0.542      1096

    accuracy                          0.711      3846
   macro avg      0.696     0.700     0.695      3846
weighted avg      0.704     0.711     0.705      3846

2
Epoch 2, Training loss: 0.6218120709154561
              precision    recall  f1-score   support

  hatespeech      0.735     0.848     0.787      1187
      normal      0.725     0.809     0.765      1563
   offensive      0.631     0.422     0.505      1096

    accuracy                          0.711      3846
   macro avg      0.697     0.693     0.686      3846
weighted avg      0.701     0.711     0.698      3846

              precision    recall  f1-score   support

  hatespeech      0.735     0.848     0.787      1187
      normal      0.725     0.809     0

In [23]:
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, classification_report

# Evaluation function with AUROC and detailed classification report
def evaluate(loader):
    model.eval()
    all_logits = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            batch = tuple(item.to(device) for item in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
            labels = batch[2]
            outputs = model(**inputs)
            logits = outputs.logits
            all_logits.extend(logits.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate probabilities
    probabilities = F.softmax(torch.tensor(all_logits), dim=1).numpy()

    # Calculate AUROC for each class (assuming binary or multi-class classification)
    if probabilities.shape[1] == 2:  # Binary classification
        auroc = roc_auc_score(all_labels, probabilities[:, 1])
    else:  # Multi-class classification
        auroc = roc_auc_score(all_labels, probabilities, multi_class="ovr")
    print(f"AUROC: {auroc}")

    # Print the classification report with 3 decimal places
    report = classification_report(all_labels, np.argmax(probabilities, axis=1), target_names=label_encoder.classes_, digits=3)
    print(report)

# Note: Make sure to call evaluate(val_loader) to see the results during or after training.
evaluate(val_loader)

AUROC: 0.870181520146165
              precision    recall  f1-score   support

  hatespeech      0.735     0.848     0.787      1187
      normal      0.725     0.809     0.765      1563
   offensive      0.631     0.422     0.505      1096

    accuracy                          0.711      3846
   macro avg      0.697     0.693     0.686      3846
weighted avg      0.701     0.711     0.698      3846



/tmp/ipykernel_34/3765978545.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  probabilities = F.softmax(torch.tensor(all_logits), dim=1).numpy()


In [24]:
model.save_pretrained('./final_fine_tuned_bert')


In [25]:
!zip -r bert_cf_model.zip /kaggle/working/final_fine_tuned_bert

  adding: kaggle/working/final_fine_tuned_bert/ (stored 0%)
  adding: kaggle/working/final_fine_tuned_bert/model.safetensors (deflated 7%)
  adding: kaggle/working/final_fine_tuned_bert/config.json (deflated 51%)
